# OpenStreetMap Data Wrangling

*By Piyush Goyal*

Map Area : [Los Angeles, California](https://s3.amazonaws.com/metro-extracts.mapzen.com/los-angeles_california.osm.bz2)

## Problems :


1. Section 1: [Data Auditing and Data Cleaning](#Section-1-:-Data-auditing-and-data-cleaning)
  * [Tag and keys count](#1.1-Tags-and-keys-counts)
  * [Special 'tag' keys values](#1.2-Special-'tag'-keys-values)
  * [Multiple Zip Codes](#1.3-Multiple-Zip-Codes)
  * [Phone Numbers](#1.4-Phone-Numbers)
  * [Street Names Abbreviated](#Street-Names-Abbreviated)


2. Section 2: [Data Overview](#Section-2:-Data-Overview)
  * [File Sizes](#File-Sizes)
  * [Number of Documents](#Number-of-Documents)
  * [Number of nodes and ways](#Number-of-nodes-and-ways)
  * [Number of distinct users](#Number-of-distinct-users)
  * [Top 10 Contributing Users](#Top-10-Contributing-Users)
  * [Number of users contributing only once](#Number-of-users-contributing-only-once)


3. Section 3: [Additional Ideas](#Section-3:-Additional-Ideas)
  * [Contributor statistics](#Contributor-statistics)
  * [Conclusion](#Conclusion)

### Section 1 : Data auditing and data cleaning

In [2]:
# coding=utf8
# -*- coding: utf8 -*-

In [3]:
# imports
import json
import xml.etree.cElementTree as ET
import codecs
import pymongo
import re
from collections import defaultdict
from pprint import pprint
import operator
import phonenumbers
import pymongo

In [4]:
# required variable declarations
__JSONFILE__ = 'los-angeles_california.osm.json'
__OSMFILE__ = 'los-angeles_california.osm'

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
address_regex = re.compile(r'^addr\:')
street_regex = re.compile(r'^street')
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

#### 1.1 Tags and keys counts

In [4]:
def count_tags(osmfile):
    tag_counts = defaultdict(int)
    tag_keys = defaultdict(int)
    
    for _, element in ET.iterparse(osmfile, ('start',)):
        tag_counts[element.tag] += 1
        
        if element.tag == 'tag' and 'k' in element.attrib: 
            tag_keys[element.get('k')] += 1

        element.clear()
        
    return tag_counts, tag_keys

In [ ]:
tag_counts, tag_keys = count_tags(__OSMFILE__)

# to print the count of each tag present
pprint(dict(tag_counts))

In [6]:
# to print the count of each key in 'tag' tag
pprint(sorted(tag_keys.items(), key = operator.itemgetter(1))[::-1][:20])

[('building', 670742),
 ('highway', 566014),
 ('ele', 484618),
 ('height', 470715),
 ('lacounty:bld_id', 469648),
 ('lacounty:ain', 469468),
 ('start_date', 443346),
 ('building:units', 422001),
 ('name', 380151),
 ('tiger:county', 243155),
 ('tiger:cfcc', 242914),
 ('source', 226421),
 ('tiger:name_base', 225098),
 ('tiger:name_type', 203989),
 ('tiger:zip_left', 185387),
 ('tiger:zip_right', 179588),
 ('tiger:reviewed', 121028),
 ('tiger:source', 112912),
 ('tiger:tlid', 112329),
 ('tiger:separated', 106620)]


From above we see the top 20 of the most frequent keys occuring in 'tag' tags.

In [7]:
def write_key_data(osmfile, tag_keys):
    """
    Parses osm file and writes the maximum of 20 data of each tag with kay-value pairs into json file
    """
    data = defaultdict(set)
    
    tag_keys = dict(tag_keys).keys()
    for _,element in ET.iterparse(osmfile, ('start',)):
        if element.tag == 'note' or element.tag == 'way':
            
            for tag in element.iter('tag'):
                if 'k' in tag.attrib and 'v' in tag.attrib:
                    key = tag.get('k')
                    if key in tag_keys and len(data[key]) < 20:
                        data[key].add(tag.get('v'))
        element.clear()
    
    #converting data so that it is json serializable
    for key in data.keys():
        data[key] = list(data[key])
    
    with open(osmfile+'-tag-key-data.json','w') as fo:
        fo.write(json.dumps(data, indent=4)+"\n")

In [8]:
write_key_data(__OSMFILE__,tag_keys)

After getting the count of key occuring, to know what each key hold and what the value means, I wrote maximum of 20 values of each key into a json file. This gave insight of some inconsistant data format (like 40mph vs 40).

#### 1.2 Special 'tag' keys values

In [9]:
unique_keys_count = 0
for key in tag_keys.keys():
    if tag_keys[key] == 1:
        unique_keys_count += 1

In [10]:
print unique_keys_count

704


So there were **704** tags with count equal 1 from which some were very specific while some were not necessary. So I manually made list of keys which are to be filter out before populating the databasewhich also contained some key-value tags with count more than 1 which has redundant data.

#### 1.3 Multiple Zip Codes

On observing the data, the zip codes were mapped to multiple keys (eg. tiger:zip_left, tiger:zip_right, etc) and also in all some of the keys multiple zip codes were present with ';' as a delimeter.

#### 1.4 Phone Numbers

Phone numbers were formatted inconsistantly. It contained values like 'yes', so all these values were ignored and the all phone numbers were reformatted to standard form (951) 587 2505 using [phonenumbers](https://pypi.python.org/pypi/phonenumbers) module in python

In [11]:
expected = ["Rapid", "Ridge", "Hills", "Bottom", "Union", "Brook", "Causeway", "Heights", "Station", "Hill",
            "Branch", "Lodge", "Rue", "Isle", "Burg", "River", "Mews", "Views", "Dam", "Cove", "Brooks",
            "Motorway", "Ways", "Throughway", "Road", "Ridges", "Path", "Haven", "Key", "Island", "Camp", 
            "Extensions", "Pine", "Ferry", "Passage", "Ports", "Spurs", "Ville", "Forges", "Valleys", "Creek",
            "Terrace", "Alley", "Course", "Prairie", "Corner", "Mill", "Glen", "Arcade", "Mills", "Plains",
            "Rest", "Bypass", "Circle", "Walk", "Circles", "Flat", "Junction", "Extension", "Park", "Lakes",
            "Ford", "Orchard", "Grove", "Courts", "Fords", "Walks", "Cape", "Landing", "Spur", "Fort",
            "Greens", "Harbor", "Light", "Plaza", "Coves", "Mall", "Green", "Islands", "Loop", "Overpass",
            "Square", "Meadow", "Stream", "Point", "Pines", "Viaduct", "Shore", "Shoal", "Pass", "Roads",
            "Place", "Glens", "Centers", "Row", "Mountain", "Boulevard", "Underpass", "Inlet", "Bayoo",
            "Lights", "Unions", "Forest", "Way", "Knoll", "Court", "Stravenue", "Crossroad", "Crest",
            "Meadows", "Tunnel", "Dale", "Mountains", "Keys", "Parkways", "Drives", "Trail", "Skyway", 
            "Pike", "Lane", "Land", "Center", "Corners", "Mount", "Ramp", "Points", "Shores", "Turnpike",
            "Flats", "Parkway", "Plain", "Loaf", "Divide", "Wall", "Streets", "Hollow", "Locks", "Canyon",
            "Spring", "Oval", "Avenue", "Junctions", "Vista", "Burgs", "Valley", "Lake", "Parks", "Field",
            "Forge", "Expressway", "Beach", "Forks", "Harbors", "Neck", "Squares", "Garden", "Manor",
            "Annex", "Club", "Villages", "Bluffs", "Wells", "Falls", "Highway", "Bluff", "Fall", "Cliff",
            "Fork", "Knolls", "Fields", "Route", "Ranch", "Curve", "Drive", "Trafficway", "Freeway",
            "Groves", "Estates", "Common", "Radial", "Crossing", "Run", "Summit", "Cliffs", "Manors",
            "Gardens", "View", "Bridge", "Trace", "Estate", "Rapids", "Well", "Lock", "Shoals",
            "Mission", "Port", "Track", "Street", "Bend", "Village", "Springs", "Crescent", "Gateway"]

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
        elem.clear()
    osm_file.close()
    return street_types

def test():
    st_types = audit(__OSMFILE__)
    pprint(dict(st_types))
    print len(st_types)

if __name__ == '__main__':
    test()

{'100': set(['3347 Michelson Dr Suite 100']),
 '102': set(['Park Point #102']),
 '103': set(['Ford  Street, Ste. 103', 'Vista Ave, Suite 103']),
 '104': set(['4121 Brockton Ave., Suite 104', 'Ventura Boulevard #104']),
 '106': set(['South Lake Ave #106', 'Ventura Boulevard #106']),
 '1101': set(['15th Street Ste. 1101']),
 '110402': set(['9th St, Unit 110402']),
 '115': set(['4th Street Suite 115']),
 '118': set(['Ventura Boulevard #118']),
 '1180': set(['Santa Monica Boulevard, Suite 1180']),
 '119': set(['North Topanga Canyon Boulevard #119', 'Ventura Boulevard #119']),
 '12': set(['Telephone Ave Suite 12']),
 '120': set(['Ventura Blvd #120']),
 '1216': set(['Laurel Canyon Blvd. #1216']),
 '138': set(['14757 State Hwy 138',
             'CA 138',
             'N. Highway 138',
             'State Highway 138']),
 '14': set(['20100 West San Fernando Road (1260) E Of Sr 14',
            '20516 West San Fernando Road W Of Sr 14',
            '23610 West San Fernando Road 1/2 Mi W Of Sr 

#### Street Names Abbreviated

There was too much inconsistancy in street names. So I changed all the street names to titliesed for (avenue -> Avenue). Also all '.', ',' characters were removed. Also street names like 'Telephone Ave Suite 12' are changed to 'Telephone Avenue Suite 12'.

In [4]:
# defining varaibles

CREATED_ATTRIBUTES = ['version', 'changeset', 'timestamp', 'user', 'uid']
POSITION_ATTRIBUTES = ['lat', 'lon']
IGNORED_TAGS = ['gnis:ST_num', 'text', 'tiger:Name', 'gnis:id', 'is_in',
    'gnis:feature_type', 'lake:surface_area:acres', 'gnis:county_id', 'iata',
    'stop', 'trees', 'icao', 'gnis:County', 'gnis:county_num', 'name:en', 'gnis:state_id',
    'health_specialty:palliative_medicine', 'tiger:STATEFP', 'name:ru', 'name:uk'
    'wikipedia:en', 'Hardware Store', 'isced', 'reg_ref', 'start_date',
    'reg_name', 'al', 'isced:level', 'source:maxspeed', 'gnis_state_id',
    'undefined', 'int_ref', 'source:ref:note', 'gnis:ST_alpha', 'gnis:feature_id',
    'practice', 'lake:shore_length:miles', 'gnis:edited', 'gnis:freature_id',
    'name:ar', 'cycleway:left', 'import_uuid', 'odbl:note', 'is_in:state',
    'gnis:reviewed', 'name:backward', 'gnis:fcode', 'is_in:country_code',
    'is_in:iso_3166_2', 'name:brand', 'name:pl', 'gnis:st_alpha']

ALIAS_TAGS = ['name_1', 'old_name', 'alt_name', 'name_2', 'place_name', 'loc_name',
    'official_name', 'name_3', 'short_name', 'bridge_name']

ZIPCODE_TAGS = ['addr:postcode', 'tiger:zip_left', 'tiger:zip_left_1', 'tiger:zip_left_2',
    'tiger:zip_left_3', 'tiger:zip_left_4', 'tiger:zip_right', 'tiger:zip_right_1',
    'tiger:zip_right_2', 'tiger:zip_right_3', 'tiger:zip_right_4']

MAPPED_TAGS = {'cosntruction': 'construction', 'construciton': 'construction',
    'EXit_to': 'exit_to', 'note:ref': 'comment', 'source:note': 'source',
    'exit_to:left': 'exit_to', 'exit_to:right': 'exit_to', 'phone': 'contact:phone',
    'maxspeed:forward': 'maxspeed'}

STREET_MAPPING = {
           "St": "Street",
           "St.": "Street",
           "ST": "Street",
           "Sreet" : "Street",
           "street" : "Street",
           "Str" : "Street",
           "road" : "Road",
           "Roadb" : "Road",
           "Rd": "Road",
           "Rd.": "Road",
           "ct." : "Court",
           "Ct" : "Court",
           "Cir" : "Circle",
           "court" : "Court",
           "Ave":"Avenue",
           "Av" :"Avenue",
           "Ave":"Avenue",
           "Ave,":"Avenue",
           "Ave.":"Avenue",
           u"Ave\u200e":"Avenue",
           "ave":"Avenue",
           "avenue":"Avenue",
           "Blv" : "Boulevard",
           "Blvd" : "Boulevard",
           "blvd" : "Boulevard",
           "Blvd." : "Boulevard",
           "BLVD." : "Boulevard",
           "Bd." : "Boulevard",
           "Bl" : "Boulevard",
           "Bl." : "Boulevard",
           "Dr" : "Drive",
           "Dr.": "Drive",
           "Ln" : "Lane",
           "Ln.": "Lane",
           "PL" : "Plaza",
           "Pl" : "Plaza",
           "Pkwy" : "Parkway",
           "Pkwy." : "Parkway",
           "Pky" : "Parkway",
           "Prkwy" : "Parkway",
           "Rte" : "Route",
           "Trails" : "Trail",
           "Trl" : "Trail",
           "way" : "Way",
           "WAY" : "Way",
           "Wy" : "Way",
           "Vw" : "View",
           "Valle" : "Valley"
}

In [83]:
def shape_element(element):
    node = {}
    created_attributes = CREATED_ATTRIBUTES
    position_attributes = POSITION_ATTRIBUTES
    ignored_tags = IGNORED_TAGS
    alias_tags = ALIAS_TAGS
    zipcode_tags = ZIPCODE_TAGS
    mapped_tags = MAPPED_TAGS
    mapping = STREET_MAPPING

    if element.tag == 'node' or element.tag == 'way':
        
        node['type'] = element.tag

        address = {}
        zipcodes = set()

        for attribute in element.attrib:
            
            if attribute in created_attributes:
                
                if 'created' not in node:
                    node['created'] = {}
                    
                node['created'][attribute] = element.get(attribute)
            elif attribute in position_attributes:
                continue
            else:
                node[attribute] = element.get(attribute)

        if 'lat' in element.attrib and 'lon' in element.attrib:
            node['pos'] = [float(element.get('lat')), float(element.get('lon'))]

        for child in element:

            if child.tag == 'nd':
                if 'node_refs' not in node:
                    node['node_refs'] = []
                if 'ref' in child.attrib:
                    node['node_refs'].append(child.get('ref'))

            if child.tag != 'tag' or 'k' not in child.attrib or 'v' not in child.attrib:
                continue
            key = child.get('k').lower()
            val = child.get('v')

            # skip problematic characters
            if problemchars.search(key):
                continue

            # skip ignored tags
            if key in ignored_tags:
                continue

            # swap keys for corrections
            if key in mapped_tags:
                key = mapped_tags[key]

            if key in zipcode_tags:
                for zipcode in process_zipcode(val):
                    zipcodes.add(zipcode)

            # set all states to CA
            if key == 'addr:state':
                key = 'CA'

            # fix and standardize phone numbers using phonenumbers module and list comprehensions
            if key == 'contact:phone':
                phone_number_matches = [m.number for m in phonenumbers.PhoneNumberMatcher(val, "US")]
                val = ';'.join([phonenumbers.format_number(phone_number_match,
                    phonenumbers.PhoneNumberFormat.NATIONAL)
                    for phone_number_match in phone_number_matches])

            if address_regex.search(key):
                key = key.replace('addr:', '')
                address[key] = val
                continue

            if key in alias_tags:
                if 'aliases' not in node:
                    node['aliases'] = {}
                node['aliases'][key] = val
                continue

            if ':' in key:
                add_branched_item(key, val, node)
                continue

            if key not in node:
                node[key] = val

        if 'name' not in node and 'aliases' in node:
            for alias in alias_tags:
                if alias in node['aliases']:
                    node['name'] = alias
                    break

        if zipcodes:
            node['zipcodes'] = list(zipcodes)

        if len(address) > 0:
            node['address'] = {}
            street_full = None
            street_dict = {}
            street_format = ['prefix', 'name', 'type']

            for key in address:
                val = address[key]
                if street_regex.search(key):
                    if key == 'street':
                        street_full = update_name(val,mapping)
                    elif 'street:' in key:
                        street_dict[key.replace('street:', '')] = val
                else:
                    node['address'][key] = val

            if street_full:
                node['address']['street'] = street_full
            elif len(street_dict) > 0:
                unclean_street = ' '.join([street_dict[key] for key in street_format])
                node['address']['street'] = update_name(unclean_street, mapping)

        return node
    else:
        return None

def update_name(name, mapping):
    
    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        if street_type in mapping.keys():
            better_name = street_type_re.sub(mapping[street_type], name)
        
        if street_type_in_between(name,mapping.keys()):
            key = street_type_in_between(name,mapping.keys())
            better_name = name.replace(key, mapping[key])
        else:
            better_name = name
        return better_name
    else:
        return name

def street_type_in_between(val, mapping_keys):
    
    for key in mapping_keys:
        if key in val:
            return key
    return None

def add_branched_item(key, val, node):

    key_split = key.split(':')
    base = key_split.pop(0)
    remainder = ':'.join(key_split)
    if type(node) == dict:
        if len(key_split) == 0:
            node[base] = val
        else:
            if base not in node:
                node[base] = {}
            add_branched_item(remainder, val, node[base])

def process_zipcode(string):
    result = []
    groups = [group.strip() for group in string.split(';')]
    for group in groups:
        if re.match(r'\d{5}\:\d{5}', group):
            group_range = map(int, group.split(':'))
            result += list(map(str, range(group_range[0], group_range[1]+1)))
        elif re.match(r'\d{5}', group):
            result.append(group)
    return result

def process_map(file_in):
    
    file_out = '{0}.json'.format(file_in)
    with codecs.open(file_out, 'w',encoding='utf-8') as fo:
        print "Writing Starts"
        for _, element in ET.iterparse(file_in):
            
            el = shape_element(element)
            if el:
                fo.write(json.dumps(el))
            element.clear()

        print "Writing Done"

process_map(__OSMFILE__)

### Section 2: Data Overview

This section contains basic statistics about the dataset and the MongoDB queries used to gather them.

##### File Sizes

los-angeles_california.osm.json : **2.5GB** <br>
lolos-angeles_california.osm    : **2.5GB**

In [71]:
connection = pymongo.Connection('localhost',27017)
db = connection.OpenMapStreet
coll = db.losAngelesCA

##### Number of Documents

In [55]:
print coll.count()

12180466


##### Number of nodes and ways

In [60]:
print 'Nodes : ',coll.find({'type' : 'node'}).count()
print 'Way : ',coll.find({'type' : 'way'}).count()

Nodes :  10955705
Way :  1224761


##### Number of distinct users

In [64]:
print len(coll.distinct('created.user'))

3301


##### Top 10 Contributing Users

In [72]:
pprint(coll.aggregate([
        {'$group' : {'_id' : '$created.user', 'count' : {'$sum' : 1}}},
        {'$sort' : {'count' : -1}},
        {'$limit' : 10}
    ]))

{u'ok': 1.0,
 u'result': [{u'_id': u'schleuss_imports', u'count': 1054163},
             {u'_id': u'manings_labuildings', u'count': 635592},
             {u'_id': u'The Temecula Mapper', u'count': 541484},
             {u'_id': u'woodpeck_fixbot', u'count': 514697},
             {u'_id': u'AM909', u'count': 447961},
             {u'_id': u'kingrollo_imports', u'count': 434711},
             {u'_id': u'nmixter', u'count': 318435},
             {u'_id': u'N76_import', u'count': 314432},
             {u'_id': u'SJFriedl', u'count': 263802},
             {u'_id': u'calfarome_labuilding', u'count': 248838}],
 u'waitedMS': 0L}


##### Number of users contributing only once

In [67]:
coll.aggregate([
        {'$group' : {'_id' : '$created.user', 'count' : {'$sum' : 1}}},
        {'$group' : {'_id' : '$count', 'num_users' : {'$sum' : 1}}},
        {'$sort' : {'_id' : 1}},
        {'$limit' : 1}
    ])

{u'ok': 1.0, u'result': [{u'_id': 1, u'num_users': 688}], u'waitedMS': 0L}

### Section 3: Additional Ideas

##### Contributor statistics
                                                
* Top user contribution percentage (“schleuss_imports”) : **8.65%** <br>
* Combined top 2 users' contribution (“schleuss_imports” and “manings_labuildings”) : **13.87%**<br>
* Combined Top 10 users contribution : **39.19%**<br>
* Combined contribution of users contributing only once - **0.006%**<br>


##### Conclusion

After this review of the data it’s obvious that the **Los Angeles California** area is incomplete, though I believe it has been well cleaned for the purposes of this exercise.